# Neuron firing fake data generating.

In [2]:
# Our numerical workhorses
import numpy as np
import scipy as sp
import pandas as pd

# Import matplotlib stuff for plotting
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Seaborn, useful for graphics
import seaborn as sns

# favorite Seaborn settings for notebooks
rc={'lines.linewidth': 2, 
    'axes.facecolor' : 'F4F3F6',
    'axes.edgecolor' : '000000',
    'axes.linewidth' : 1.2,
    'grid.color' : 'a6a6a6',
    'lines.linewidth': 2,
    'axes.labelsize': 18,
    'axes.titlesize': 20,
    'xtick.major' : 13,
    'xtick.labelsize': 'large',
    'ytick.labelsize': 13,
    'font.family': 'Lucida Sans Unicode',
    'grid.linestyle': ':',
    'grid.linewidth': 1.5,
    'mathtext.fontset': 'stixsans',
    'mathtext.sf': 'sans',
    'legend.frameon': True,
    'legend.fontsize': 13}

plt.rc('text.latex', preamble=r'\usepackage{sfmath}')
plt.rc('mathtext', fontset='stixsans', sf='sans')
sns.set_style('darkgrid', rc=rc)
sns.set_palette("colorblind", color_codes=True)
sns.set_context('notebook', rc=rc)

# Setting up the problem

In this notebook we will try to simulate fake data for neurons firing with certain rate where the probability distribution of the $N$ neurons firing is of the form
$$
P(\{ \sigma_i \}) = {e^{- \beta \mathcal{H}(\{ \sigma_i \})} \over \mathcal{Z}},
$$
where $\sigma_i \in \{0, 1\}$, with $i \in \{1,\ldots, N\}$ is a spin-like variable that determines if the $i^{th}$ neuron fired in a time interval $\Delta t \sim 40$ ms. This means that if during the time interval the $i^{th}$ neuron fires the variable $\sigma_i$ takes the value of $1$, or zero otherwise.

The Hamiltonian of this distirbution is of the form
$$
\mathcal{H}(\{ \sigma_i \}) = - \sum_i h_i \sigma_i 
- {1 \over 2} \sum_{i, j} J_{ij} \sigma_i \sigma_j,
$$
where $h_i$ is related to the intrinsic firing probability of a neuron, and $J_{ij}$ is related to the coupling of neurons $i$ and $j$. Finally $\mathcal{Z}$ serves as a normalization constant.

# Monte Carlo simulation of the neuron firing

Given that for 10 neurons we already have 2^{10} possible states for the system it is not tracrtable to generate the full distribution. Instead we will follow a Monte Carlo sampling of the distribution. The algorithm will work as follow:
1. Initialize the vector $\{ \sigma_i \}$ at a random configuration.
2. Choose one of the spins to be flipped and compute the energy difference $\Delta E$ as given by the Hamiltonian between both states.
3. If The energy change $\Delta E < 0$ accept the step with probability 1.
4. If the energy change $\Delta E \geq 0$ then generate a random number between 0 and 1. If the Boltzmann weight $e^{- \Delta E}$ is greater than this random number accept the change.
5. Repeat from step 2.

Let's define a function that given a vector $\{\sigma_i\}$ and a vector of magnetic fields $\{h_i \}$ and a coupling matrix $\mathbf{J}$ computes the Hamiltonian $\mathcal{H}$.

In [3]:
def ising_hamiltonian(sigma_array, h_array, J_mat):
    '''
    Computes the hamiltonian H({sigma_i}) for a vector sigma_array given a
    vector of magnetic fields h_array and a coupling matrix J_mat.
    Parameters
    ----------
    sigma_array : array-like.
        Array with neuron states (0 or 1).
    h_array : array-like. len(h_array) = len(sigma_array)
        Array with the values of the external magnetic field for each 
        of the neurons.
    J_mat : 2D-array. shape = len(sigma_array) x len(sigma_array)
        Matrix with the coupling values between neurons.
    Returns
    -------
    Hamiltonian value for state sigma_array.
    '''
    return np.dot(sigma_array) + np.sum(np.dot(J_mat, sigma_array))

For the sake of the simulation let's define generate an array of $\{h_i \}$ coupling matrix $\mathbf{J}$ given a truncated Gaussian distribution with a given mean and variance.

In [11]:
# Define the number of cells
n_cells = 10

# Generate the coupling matrix given a Gaussian distribution
J_mat = np.random.normal(loc=0, scale=1, size=[n_cells, n_cells])

# Generate the array of external magnetic fields
h_array = - np.random.exponential(1, size=n_cells)

Let's now define the Monte Carlo simulation function.

In [ ]:
def ising_monte_carlo(sigma_0, h_array, J_array, n_steps=1000):
    '''
    Function to perform a Monte Carlo simulation of the neurons firing giving
    an initial condition sigma_0, an external magnetic field h_array and a
    coupling matrix J_array.
    Parameters
    ----------
    sigma_0 : array-like.
        Initial condition of the neuronal state
    h_array : array-like. len(h_array) = len(sigma_array)
        Array with the values of the external magnetic field for each 
        of the neurons.
    J_mat : 2D-array. shape = len(sigma_array) x len(sigma_array)
        Matrix with the coupling values between neurons.
    n_steps : int.
        Number of steps to perform in the simulation.
    '''
    # Define the number of cells in simulation
    n_cells = len(sigma_0)
    
    